In [31]:
import copy
import itertools
import os
import time
from tempfile import TemporaryDirectory


# 3rd party modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import timm
import torch
import torchvision
from PIL import Image
import pickle
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset, Subset
from torch.quantization import prepare, convert
from torch.quantization import fuse_modules, QuantStub, DeQuantStub
from torchvision import datasets, models, transforms
from torchvision.io import read_image
from torch.quantization import quantize_dynamic
from tqdm import tqdm

Step 1: Load the Trained Model


In [32]:
def initialize_model(output_classes=6, quantize = True):
    """
    Initializes the EfficientNet-B5 model with a custom final layer for the given number of output classes.

    Parameters:
    - output_classes (int): The number of classes for the final output layer.

    Returns:
    - model (torch.nn.Module): The modified model based on the pre-trained EfficientNetB5 feature representation.
    """

    # Load a pre-trained EfficientNet-B5 model from the timm library
    model = timm.create_model('efficientnet_b5', pretrained=True)

    # # Freeze all the parameters in the model to prevent them from being updated during training
    # for param in model.parameters():
    #     param.requires_grad = False
    ## This was removed in part from ablation study, because the water bottle classification was making plastic heavy in prediction for the model.


    # Get the number of input features to the final fully connected layer
    # The classifier layer is the final layer in EfficientNet models
    in_features = model.classifier.in_features

    # Replace the final classifier layer with a new one that has the desired number of output classes
    model.classifier = nn.Sequential(
        nn.Linear(in_features, 512),  # Reduce dimension from in_features to 512 (from 1280 in this case)
        nn.ReLU(),                    # Apply ReLU activation function
        nn.Linear(512, output_classes)  # Final layer with 'output_classes' number of outputs
    )

    if quantize:
        model.quant = QuantStub()
        model.dequant = DeQuantStub()

    return model

def fuse_model(model):
    """
    Applies module fusion to optimize layers of a given model for quantization. Fuses Convolution, BatchNorm,
    and ReLU layers where applicable.

    Parameters:
    - model (torch.nn.Module): The model to be fused.

    Returns:
    - model (torch.nn.Module): The model with fused layers.
    """
    for module_name in ["layer1", "layer2", "layer3", "layer4"]:
        module = getattr(model, module_name)
        for submodule_name, submodule in module.named_children():
            if 'conv' in submodule_name:
                fuse_modules(submodule, ['conv1', 'bn1', 'relu'], inplace=True)
                if hasattr(submodule, 'downsample'):
                    fuse_modules(submodule.downsample, ['0', '1'], inplace=True)
    return model

class QuantizedResNet(nn.Module):
    """
    A wrapper class for the quantized ResNet model that includes quantization and dequantization steps in the forward pass.

    Attributes:
    - model (torch.nn.Module): The base model with added quantization and dequantization modules.
    """
    def __init__(self, model):
        super(QuantizedResNet, self).__init__()
        self.model = model
    
    def forward(self, x):
        """
        Processes input through the model's quantization, base layers, and dequantization.

        Parameters:
        - x (Tensor): The input tensor.

        Returns:
        - x (Tensor): The output tensor after processing.
        """
        x = self.model.quant(x)
        x = self.model(x)
        x = self.model.dequant(x)
        return x

def static_quantize(model, dataloader, device):
    """
    Applies static quantization to a model. This process includes inserting observers,
    calibrating the model with a calibration dataset, and converting the model to use quantized weights.

    Parameters:
    - model (torch.nn.Module): The model to quantize.
    - dataloader (DataLoader): The DataLoader for the calibration dataset.
    - device (torch.device): The device to perform quantization on.

    Returns:
    - model_int8 (torch.nn.Module): The quantized model.
    """
    model.to(device)
    model.eval()
    model_fp32_prepared = prepare(model)
    with torch.no_grad():
        for batch in dataloader:
            inputs, _ = batch
            inputs = inputs.to(device)
            model_fp32_prepared(inputs)
    model_int8 = convert(model_fp32_prepared)
    return model_int8

def load_data_from_pickle(file_path):
    """
    Loads image paths and labels from a pickle file.

    Parameters:
    - file_path (str): The path to the pickle file containing the data.

    Returns:
    - tuple: A tuple containing lists of image paths and their corresponding labels.
    """
    with open(file_path, 'rb') as file:
        image_paths, labels = pickle.load(file)
    return image_paths, labels

def load_model(model_path, output_classes=6):
    """
    Loads a trained model from a specified file path.

    Parameters:
    - model_path (str): The path to the file containing the model's state dictionary.
    - output_classes (int): The number of output classes for the model's fully connected layer.

    Returns:
    - model (torch.nn.Module): The loaded and initialized model.
    """
    model = initialize_model(output_classes)
    saved_contents = torch.load(model_path)
    state_dict = saved_contents["state_dict"]
    model.load_state_dict(state_dict)
    return model

In [33]:
class CustomDataset(Dataset):
    """
    A custom dataset class that extends PyTorch's Dataset class for image loading and preprocessing.

    Attributes:
    - image_paths (list): List of paths to the images.
    - labels (list): List of labels corresponding to the images.
    - transform (callable, optional): Optional transform to be applied on a sample.
    """

    def __init__(self, image_paths, labels, transform=None):
        """
        Initializes the dataset with images and labels.

        Parameters:
        - image_paths (list): List of paths to the images.
        - labels (list): List of labels for the images.
        - transform (callable, optional): Optional transform to apply on images.
        """

        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """

        return len(self.image_paths)

    def __getitem__(self, index):
        """
        Retrieves an image and its label from the dataset at the specified index.

        Parameters:
        - index (int): Index of the image and label to return.

        Returns:
        - tuple: A tuple containing the image and its label.
        """

        image_path = self.image_paths[index]
        # Load the image as a PIL Image
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[index], dtype=torch.long)
        return image, label

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Step 2: Apply Quantization


In [ ]:
# Set the device for computation - Quantized models are typically used on CPUs
device = torch.device("cpu")

# Load test data from a serialized pickle file
test_file = 'full_dataset_segmented_test_data.pkl'
test_paths, test_labels = load_data_from_pickle(test_file)

# Normalization parameters aligned with those used for pre-trained models
normalization = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

# Transformation pipeline for testing data, suitable for EfficientNet and other ImageNet-trained models
transform = transforms.Compose([
    transforms.Resize(224, antialias=True),  # Resize images to 224x224 pixels, applying antialiasing for image quality
    transforms.ToTensor(),                   # Convert images to tensor format suitable for model input
    transforms.Normalize(*normalization),    # Apply normalization using predefined mean and std deviation values
])

# Create a dataset and dataloader for testing
test_dataset = CustomDataset(test_paths, test_labels, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=0)

# Load a pre-trained model from a specified path and prepare it for quantization
model_path = 'RESNET50_best_model.pth'  # Model path could be adjusted if needed
model = load_model(model_path)

# Apply static quantization to the model
quantized_model = static_quantize(QuantizedResNet(model), test_dataloader, device)
quantized_model.to(device)  # Move the quantized model to the designated computing device

Step 3: Evaluate the Quantized Model


In [39]:
def evaluate_model(model, dataloader, loss_fun, device):
    """
    Evaluates the model's performance on a given dataset.

    Parameters:
    - model: The neural network model to evaluate.
    - dataloader: The DataLoader containing the dataset for evaluation.
    - loss_fun: The loss function used to compute the model's loss.
    - device: The device (CPU or CUDA) on which the computations will be performed.

    Returns:
    - A tuple of average loss and accuracy over the dataset.
    """

    model.eval()  # Set the model to evaluation mode    test_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    with torch.no_grad():  # No gradients needed for evaluation
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)  # Move data to the same device as the model

            pred = model(x)
            _, predicted_classes = torch.max(pred, 1)
            correct_predictions = (predicted_classes == y).float()

            loss = loss_fun(pred, y.long())  # Ensure consistent data type
            val_loss += loss.item()
            val_acc += correct_predictions.sum().item() / y.size(0)

        val_loss /= len(dataloader)
        val_acc /= len(dataloader)

    return val_loss, val_acc

In [40]:
# Evaluate the model
loss_fun = nn.CrossEntropyLoss()
test_loss, test_acc = evaluate_model(quantized_model, test_dataloader, loss_fun, device)

print(f"Quantized Model Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Quantized Model Test Loss: 0.5621, Test Accuracy: 0.8341
